In [7]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

def process_single_angle_data(data, num_frames=300, num_joints=17):
    # Ensure data is numpy array
    data = np.array(data)
    
    # Normalize the data
    max_val = np.max(data)
    min_val = np.min(data)
    normalized_data = (data - min_val) / (max_val - min_val)
    
    # Reshape to [batch_size, num_channels, num_frames, num_joints]
    data = np.transpose(normalized_data, (0, 3, 1, 2))
    
    # If you have less than num_frames, repeat the data
    if data.shape[2] < num_frames:
        data = np.repeat(data, num_frames // data.shape[2] + 1, axis=2)[:, :, :num_frames, :]
    
    # If you have more than num_frames, truncate the data
    if data.shape[2] > num_frames:
        data = data[:, :, :num_frames, :]
    
    return torch.FloatTensor(data)

class PoseDataset(Dataset):
    def __init__(self, data_0, data_90, data_180, num_frames=300):
        self.data_0 = process_single_angle_data(data_0, num_frames)
        self.data_90 = process_single_angle_data(data_90, num_frames)
        self.data_180 = process_single_angle_data(data_180, num_frames)
        
    def __len__(self):
        return len(self.data_0) + len(self.data_90) + len(self.data_180)
    
    def __getitem__(self, idx):
        if idx < len(self.data_0):
            return self.data_0[idx], 0  # 0 degree angle
        elif idx < len(self.data_0) + len(self.data_90):
            return self.data_90[idx - len(self.data_0)], 1  # 90 degree angle
        else:
            return self.data_180[idx - len(self.data_0) - len(self.data_90)], 2  # 180 degree angle

# Example usage
data_0 = [[[[38.89351272583008, 17.675010681152344], [43.859161376953125, 13.46286392211914], [35.017845153808594, 13.283355712890625], [51.712310791015625, 13.568647384643555], [29.73617935180664, 13.277813911437988], [62.442684173583984, 34.74855422973633], [19.7756404876709, 35.577674865722656], [76.11538696289062, 63.75419235229492], [5.235096454620361, 66.12895965576172], [76.3991470336914, 95.9053726196289], [4.93015718460083, 101.26876831054688], [54.767093658447266, 102.19705200195312], [27.949174880981445, 102.5954818725586], [55.661502838134766, 156.7456512451172], [27.860849380493164, 157.37513732910156], [55.00213623046875, 207.13873291015625], [31.336200714111328, 208.50503540039062]]]]

data_90 = [[[[32.15562438964844, 37.65242004394531], [37.92998504638672, 30.083906173706055], [29.943157196044922, 32.57968521118164], [58.07182693481445, 27.735185623168945], [0.0, 0.0], [78.5476303100586, 63.19194793701172], [50.73186492919922, 70.51588439941406], [84.50447082519531, 122.54889678955078], [0.0, 0.0], [66.65069580078125, 172.83421325683594], [0.0, 0.0], [82.80135345458984, 173.37745666503906], [72.25676727294922, 174.55914306640625], [36.53921127319336, 263.5566101074219], [73.65975189208984, 264.79425048828125], [26.288681030273438, 347.690673828125], [102.2801742553711, 344.6444396972656]]]]

data_180 = [[[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [44.35426330566406, 19.40471076965332], [79.07585144042969, 18.704072952270508], [28.457805633544922, 54.024662017822266], [92.90510559082031, 55.75114059448242], [8.339852333068848, 108.02925872802734], [106.04246520996094, 112.90032958984375], [7.4025983810424805, 152.83641052246094], [113.28245544433594, 160.9684295654297], [32.518184661865234, 155.5282745361328], [74.9278564453125, 155.7244415283203], [31.802139282226562, 228.9239044189453], [72.60979461669922, 230.3274383544922], [37.51327896118164, 312.47698974609375], [67.4405746459961, 315.91522216796875]]]]

# Create dataset
dataset = PoseDataset(data_0, data_90, data_180)

# Create dataloader
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

# Example of iterating through the data
for data, angle in dataloader:
    print("Data shape:", data.shape)
    print("Angle:", angle)
    break

Data shape: torch.Size([1, 2, 300, 17])
Angle: tensor([1])


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class Graph():
    def __init__(self, layout='openpose', strategy='spatial'):
        self.get_edge(layout)
        self.get_adjacency(strategy)

    def get_edge(self, layout):
        if layout == 'openpose':
            self.num_node = 18
            self_link = [(i, i) for i in range(self.num_node)]
            neighbor_link = [(4, 3), (3, 2), (7, 6), (6, 5), (13, 12), (12, 11),
                             (10, 9), (9, 8), (11, 5), (8, 2), (5, 1), (2, 1),
                             (0, 1), (15, 0), (14, 0), (17, 15), (16, 14)]
            self.edge = self_link + neighbor_link
            self.center = 1
        else:
            raise ValueError("Do not support this layout.")

    def get_adjacency(self, strategy):
        valid_hop = range(0, self.num_node - 1)
        adjacency = np.zeros((self.num_node, self.num_node))
        for hop in valid_hop:
            adjacency[self.hop_dis == hop] = 1
        normalize_adjacency = normalize_digraph(adjacency)

        if strategy == 'uniform':
            A = np.zeros((1, self.num_node, self.num_node))
            A[0] = normalize_adjacency
            self.A = A
        elif strategy == 'distance':
            A = np.zeros((len(valid_hop), self.num_node, self.num_node))
            for i, hop in enumerate(valid_hop):
                A[i][self.hop_dis == hop] = normalize_adjacency[self.hop_dis == hop]
            self.A = A
        elif strategy == 'spatial':
            A = []
            for hop in valid_hop:
                a_root = np.zeros((self.num_node, self.num_node))
                a_close = np.zeros((self.num_node, self.num_node))
                a_further = np.zeros((self.num_node, self.num_node))
                for i in range(self.num_node):
                    for j in range(self.num_node):
                        if self.hop_dis[j, i] == hop:
                            if self.hop_dis[j, self.center] == self.hop_dis[i, self.center]:
                                a_root[j, i] = normalize_adjacency[j, i]
                            elif self.hop_dis[j, self.center] > self.hop_dis[i, self.center]:
                                a_close[j, i] = normalize_adjacency[j, i]
                            else:
                                a_further[j, i] = normalize_adjacency[j, i]
                if hop == 0:
                    A.append(a_root)
                else:
                    A.append(a_root + a_close)
                    A.append(a_further)
            A = np.stack(A)
            self.A = A
        else:
            raise ValueError("Do not support this strategy.")

def normalize_digraph(A):
    Dl = np.sum(A, 0)
    num_node = A.shape[0]
    Dn = np.zeros((num_node, num_node))
    for i in range(num_node):
        if Dl[i] > 0:
            Dn[i, i] = Dl[i]**(-1)
    AD = np.dot(A, Dn)
    return AD

class ConvTemporalGraphical(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, t_kernel_size=1, t_stride=1, t_padding=0, t_dilation=1, bias=True):
        super().__init__()
        self.kernel_size = kernel_size
        self.conv = nn.Conv2d(in_channels, out_channels * kernel_size, kernel_size=(t_kernel_size, 1), 
                              padding=(t_padding, 0), stride=(t_stride, 1), dilation=(t_dilation, 1), bias=bias)

    def forward(self, x, A):
        assert A.size(0) == self.kernel_size
        x = self.conv(x)
        n, kc, t, v = x.size()
        x = x.view(n, self.kernel_size, kc//self.kernel_size, t, v)
        x = torch.einsum('nkctv,kvw->nctw', (x, A))
        return x.contiguous(), A

class STGCNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, dropout=0, residual=True):
        super().__init__()
        assert len(kernel_size) == 2
        assert kernel_size[0] % 2 == 1
        padding = ((kernel_size[0] - 1) // 2, 0)
        
        self.gcn = ConvTemporalGraphical(in_channels, out_channels, kernel_size[1])
        self.tcn = nn.Sequential(
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, (kernel_size[0], 1), (stride, 1), padding),
            nn.BatchNorm2d(out_channels),
            nn.Dropout(dropout, inplace=True)
        )
        
        if not residual:
            self.residual = lambda x: 0
        elif (in_channels == out_channels) and (stride == 1):
            self.residual = lambda x: x
        else:
            self.residual = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=(stride, 1)),
                nn.BatchNorm2d(out_channels)
            )
        
        self.relu = nn.ReLU(inplace=True)
    
    def forward(self, x, A):
        res = self.residual(x)
        x, A = self.gcn(x, A)
        x = self.tcn(x) + res
        return self.relu(x), A

class STGCN(nn.Module):
    def __init__(self, in_channels, num_class, graph_args, edge_importance_weighting, **kwargs):
        super().__init__()
        self.graph = Graph(**graph_args)
        A = torch.tensor(self.graph.A, dtype=torch.float32, requires_grad=False)
        self.register_buffer('A', A)
        
        spatial_kernel_size = A.size(0)
        temporal_kernel_size = 9
        kernel_size = (temporal_kernel_size, spatial_kernel_size)
        
        self.data_bn = nn.BatchNorm1d(in_channels * A.size(1))
        
        self.st_gcn_networks = nn.ModuleList((
            STGCNBlock(in_channels, 64, kernel_size, stride=1, residual=False),
            STGCNBlock(64, 64, kernel_size, stride=1),
            STGCNBlock(64, 64, kernel_size, stride=1),
            STGCNBlock(64, 64, kernel_size, stride=1),
            STGCNBlock(64, 128, kernel_size, stride=2),
            STGCNBlock(128, 128, kernel_size, stride=1),
            STGCNBlock(128, 128, kernel_size, stride=1),
            STGCNBlock(128, 256, kernel_size, stride=2),
            STGCNBlock(256, 256, kernel_size, stride=1),
            STGCNBlock(256, 256, kernel_size, stride=1)
        ))
        
        if edge_importance_weighting:
            self.edge_importance = nn.ParameterList([
                nn.Parameter(torch.ones(self.A.size()))
                for i in self.st_gcn_networks
            ])
        else:
            self.edge_importance = [1] * len(self.st_gcn_networks)
        
        self.fcn = nn.Conv2d(256, num_class, kernel_size=1)
    
    def forward(self, x):
        N, C, T, V = x.size()
        x = x.permute(0, 3, 1, 2).contiguous()
        x = x.view(N, V * C, T)
        x = self.data_bn(x)
        x = x.view(N, V, C, T)
        x = x.permute(0, 2, 3, 1).contiguous()
        x = x.view(N, C, T, V)
        
        for gcn, importance in zip(self.st_gcn_networks, self.edge_importance):
            x, _ = gcn(x, self.A * importance)
        
        x = F.avg_pool2d(x, x.size()[2:])
        x = x.view(N, -1, 1, 1)
        x = self.fcn(x)
        x = x.view(x.size(0), -1)
        
        return x

class ModifiedSTGCN(nn.Module):
    def __init__(self, in_channels, num_class, graph_args, edge_importance_weighting, **kwargs):
        super().__init__()
        self.st_gcn = STGCN(in_channels, num_class, graph_args, edge_importance_weighting, **kwargs)
        self.angle_embedding = nn.Embedding(3, 64)  # 3 angles, embedding size 64
        self.final_fc = nn.Linear(num_class + 64, num_class)

    def forward(self, x, angle):
        x = self.st_gcn(x)
        angle_embed = self.angle_embedding(angle).squeeze(1)
        combined = torch.cat([x, angle_embed], dim=1)
        return self.final_fc(combined)

# Example usage
graph_args = {'layout': 'openpose', 'strategy': 'spatial'}
model = ModifiedSTGCN(in_channels=2, num_class=10, graph_args=graph_args, edge_importance_weighting=True)

# Example input
batch_size = 2
num_channels = 2  # X and Y coordinates
num_frames = 300
num_joints = 18
x = torch.randn(batch_size, num_channels, num_frames, num_joints)
angle = torch.LongTensor([0, 1])  # Example angles for each sample in the batch

# Forward pass
output = model(x, angle)
print(output.shape)  # Should be [batch_size, num_class]

AttributeError: 'Graph' object has no attribute 'hop_dis'